In [1]:
from selenium import webdriver # Importar web driver desde selenium. Web Driver = Un navegador web
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests

from bs4 import BeautifulSoup

import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from urllib.parse import urljoin

import math
import re

In [2]:
# Establecer rango de precios para las casas y apartamentos
lower_price = 400000
upper_price = 2000000
# Lista en blanco donde se almacenarán los anuncios de casas y apartamentos
house_apartment_list = []

In [3]:
def get_page_url(page_number,rango_i,rango_s):
    
    base_page_url = "https://www.encuentra24.com/guatemala-es/bienes-raices-venta-de-propiedades-casas/guatemala-es-guatemala.{}?q=f_price.{}-{}|f_currency.GTQ".format(page_number,rango_i,rango_s)
    
    return base_page_url

In [4]:
# Inicializar el driver de Firefox 
driver = webdriver.Firefox() 
# Maximizar ventana
driver.maximize_window()
# Ingresar al url
driver.get(get_page_url(1,lower_price,upper_price))
# Pausa para asegurar que la página cargue completamente (puedes ajustar el tiempo según la velocidad de carga)
time.sleep(5)
# Obtener el código HTML de la página cargada
html_code = driver.page_source
# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(html_code, 'html.parser')

In [5]:
all_house_div = soup.find_all("div", class_='d3-ad-tile')

In [21]:
house_div = all_house_div[7]

In [12]:
a = house_div.find(class_='d3-ad-tile__details').text.strip()

In [14]:
metros = a.split("\n")

In [17]:
metros[5]

'3'

In [24]:
data = pd.read_csv('7_19_2024 iter 2.csv')

FileNotFoundError: [Errno 2] No such file or directory: '7_19_2024 iter 2.csv'

In [ ]:
if icon:
            # Identificar la categoría basada en el ícono
            icon_use = icon.find('use')
            if icon_use:
                icon_id = icon_use.get('xlink:href')
                if 'resize' in icon_id:  # Metros cuadrados
                    square_meters = detail.text.strip().split()[0]
                elif 'bed' in icon_id:  # Habitaciones
                    rooms = detail.text.strip()
                elif 'parking' in icon_id:  # Parqueos
                    parking = detail.text.strip()
                elif 'bath' in icon_id:  # Baños
                    baths = detail.text.strip()

        
# Imprimir los resultados de este anuncio
    print("Metros cuadrados:", square_meters if square_meters else "No especificado")
    print("Habitaciones:", rooms if rooms else "No especificado")
    print("Parqueos:", parking if parking else "No especificado")
    print("Baños:", baths if baths else "No especificado")
    print("---------------------------------------------")


In [22]:
def div_html_to_obj(house_div):
    # Intentamos primero buscar el atributo data-original para la imagen, si no existe, buscamos src
    try:
        img_url = house_div.find("img")['data-original']
    except:
        img_url = house_div.find("img")['src']  

    location = house_div.find(class_='d3-ad-tile__location').text.strip()

    title = house_div.find(class_='d3-ad-tile__title').text
    
    # Obtenemos el elemento que contiene el precio
    price_dirt = house_div.find(class_="d3-ad-tile__price").text

    # Definimos una expresión regular para extraer el precio principal
    price_regex = r'([Q$]\d[\d,.]+)'

    # Buscamos el precio
    original_price = re.findall(price_regex, price_dirt)

    # Obtenemos el texto de los atributos
    attributes = house_div.find(class_='d3-ad-tile__details').find_all('li', class_='d3-ad-tile__details-item')

    # Inicializamos las variables
    square_meters = None
    rooms = None
    parking = None
    baths = None

    # Iteramos sobre los detalles para extraer la información
    for attribute in attributes:
        icon = attribute.find('svg', class_='d3-icon d3-ad-tile__icon')
        value = attribute.text.strip()

        if icon:
            icon_use = icon.find('use').get('xlink:href')

            if 'resize' in icon_use:  # Metros cuadrados
                square_meters = int(re.search(r'\d+', value).group())
            elif 'bed' in icon_use:  # Habitaciones
                rooms = int(value)
            elif 'parking' in icon_use:  # Parqueos
                parking = int(value)
            elif 'bath' in icon_use:  # Baños
                baths = int(value)

    url = 'https://www.encuentra24.com' + house_div.find("a")["href"]

    return {
        "title": title,
        "price": original_price,
        "square_meters": square_meters,
        "rooms": rooms,
        "parking": parking,
        "baths": baths,
        "location": location,
        "img_url": img_url,
        "url": url
    }

In [23]:
div_html_to_obj(house_div)

{'title': ' Casa en venta en Santa Rita zona 2 de Mixco',
 'price': ['Q1,800,000'],
 'square_meters': 210,
 'rooms': 5,
 'parking': None,
 'baths': 4,
 'location': 'Zona 2',
 'img_url': 'https://photos.encuentra24.com/t_or_fh_m/f_auto/v1/gt/27/99/24/52/27992452_fea9b2',
 'url': 'https://www.encuentra24.com/guatemala-es/bienes-raices-venta-de-propiedades-casas/casa-en-venta-en-santa-rita-zona-2-de-mixco/27992452?q=f_price.400000-2000000|f_currency.GTQ&regionslug=guatemala-es-guatemala&list=categoryregion&categoryslug=bienes-raices-venta-de-propiedades-casas'}